# 4. Manipulates Raw Statistics into Multiple Forms #
## For Brownlow Predictor Project ##

Turns raw counts of game/player stats into
- [x] Standardised
- [x] Normalised
- [x] Rank followed by Standardisation
- [x] Percentages
- [ ] Percentages followed by Standardisation       *[since proven to be exactly same as Standardisation]*
- [x] Percentages followed by Normalisation

*In order for different statistics to be used together for predictive purposes an important step is to manipulate them into a form which they are 'equal'. There are many ways to do so, and the author wanted to use different manipulations of data to see which gave the best results in terms of prediction. Also wanted to test in general the effectiveness of each data manipulation format*

***Author: `Lang (Ron) Chen` 2021.12-2022.1**

---

*Note: As Percentages followed by Standardisation was proven to be exactly the same as Standardisation, all relevent code have bben commented out* 

**0. Import Libraries**

In [1]:
import pandas as pd
import os

In [2]:
choice = 'OriginalData'

**1. Writing the functions for standardisation**

The reason that each method of manipulation contains 4 functions (except for percentage) is because of the existance of negative stats in which the scores should be reversed (i.e. turnovers, frees against). 

- BT means both teams - manipulates the data with respects to all players on ground
- OT means own teams - manipulates the data with respect to only teammates
- inv means inverse - the scores would be ranked in reverse (according to what is 'reverse' for that particular manipulated method)

'Normalisation followed by Standardisation' or vice versa were initially also planned to be used, before an experiment showed that the former is equal to 'Standardisation' whilst the latter is exactly equal to 'Normalisation'.

1. Standardisation

In [3]:
def std_BT(col):
    """ Takes in input of a dataframe column """
    
    game_max = col.max()
    game_min = col.min()
    
    return (col-game_min)/(game_max-game_min)

In [4]:
def std_BT_inv(col):
    """ Takes in input of a dataframe column """
    
    game_max = col.max()
    game_min = col.min()
    
    return 1- (col-game_min)/(game_max-game_min)

In [5]:
def std_OT(df, colname):
    """ Takes in two inputs: the first a dataframe with two columns: the stat column and the HomeAway column; the second a string which is the column name of the stat column """
    
    tmp1 = df[df['HomeAway'] == 'Home']
    tmp2 = df[df['HomeAway'] == 'Away']
    
    t1_max = tmp1[colname].max()
    t1_min = tmp1[colname].min()
    t2_max = tmp2[colname].max()
    t2_min = tmp2[colname].min()
    
    out1 = pd.DataFrame(((tmp1[colname]-t1_min)/(t1_max-t1_min)))
    out2 = pd.DataFrame(((tmp2[colname]-t2_min)/(t2_max-t2_min)))
    out = out1.append(out2)
    
    return out

In [6]:
def std_OT_inv(df, colname):
    """ Takes in two inputs: the first a dataframe with two columns: the stat column and the HomeAway column; the second a string which is the column name of the stat column """
    
    tmp1 = df[df['HomeAway'] == 'Home']
    tmp2 = df[df['HomeAway'] == 'Away']
    
    t1_max = tmp1[colname].max()
    t1_min = tmp1[colname].min()
    t2_max = tmp2[colname].max()
    t2_min = tmp2[colname].min()
    
    out1 = pd.DataFrame(1-((tmp1[colname]-t1_min)/(t1_max-t1_min)))
    out2 = pd.DataFrame(1-((tmp2[colname]-t2_min)/(t2_max-t2_min)))
    out = out1.append(out2)
    
    return out

2. Normalisation

In [7]:
def norm_BT(col):
    """ Takes in input of a dataframe column """
    
    game_mean = col.mean()
    game_std = col.std()
    
    return (col-game_mean)/game_std

In [8]:
def norm_BT_inv(col):
    """ Takes in input of a dataframe column """
    
    game_mean = col.mean()
    game_std = col.std()
    
    return -1 * (col-game_mean)/game_std

In [9]:
def norm_OT(df, colname):
    """ Takes in two inputs: the first a dataframe with two columns: the stat column and the HomeAway column; the second a string which is the column name of the stat column """
    
    tmp1 = df[df['HomeAway'] == 'Home']
    tmp2 = df[df['HomeAway'] == 'Away']
    
    t1_mean = tmp1[colname].mean()
    t1_std = tmp1[colname].std()
    t2_mean = tmp2[colname].mean()
    t2_std = tmp2[colname].std()
    
    out1 = pd.DataFrame(((tmp1[colname]-t1_mean)/t1_std))
    out2 = pd.DataFrame(((tmp2[colname]-t2_mean)/t2_std))
    out = out1.append(out2)
    
    return out

In [10]:
def norm_OT_inv(df, colname):
    """ Takes in two inputs: the first a dataframe with two columns: the stat column and the HomeAway column; the second a string which is the column name of the stat column """
    
    tmp1 = df[df['HomeAway'] == 'Home']
    tmp2 = df[df['HomeAway'] == 'Away']
    
    t1_mean = tmp1[colname].mean()
    t1_std = tmp1[colname].std()
    t2_mean = tmp2[colname].mean()
    t2_std = tmp2[colname].std()
    
    out1 = pd.DataFrame((-1*(tmp1[colname]-t1_mean)/t1_std))
    out2 = pd.DataFrame((-1*(tmp2[colname]-t2_mean)/t2_std))
    out = out1.append(out2)
    
    return out

3. Rank followed by Standardisation

In [11]:
def rank_std_BT(col):
    """ Takes in input of a dataframe column """
    
    tmp = [0 for i in range(len(col))]
    
    data = list(col)
    data_items = [(i, stat) for i, stat in enumerate(data)]
    
    data_items.sort(key = lambda x:x[1])
    
    switch = False
    for i in range(len(data_items)):
        if i+1 < len(data_items):
            if data_items[i][1] == data_items[i+1][1] and switch:
                switch = True
                start = i

            elif switch:
                switch = False

                val = (start+i)/2
                for j in range(start, i+1):
                    tmp[data_items[j][0]] = val + 1

            else:
                tmp[data_items[i][0]] = i + 1
        
        else:
            if switch:

                val = (start+i)/2
                for j in range(start, i+1):
                    tmp[data_items[j][0]] = val + 1

            else:
                tmp[data_items[i][0]] = i + 1
    
    tmp_max = max(tmp)
    tmp_min = min(tmp)
    out = [((tmp[i]-tmp_min)/(tmp_max-tmp_min)) for i in range(len(tmp))]
    
    
    return pd.DataFrame({'tmp': out})

In [12]:
def rank_std_BT_inv(col):
    """ Takes in input of a dataframe column """
    
    tmp = [0 for i in range(len(col))]
    
    data = list(col)
    data_items = [(i, stat) for i, stat in enumerate(data)]
    
    data_items.sort(key = lambda x:x[1], reverse = True)
    
    switch = False
    for i in range(len(data_items)):
        if i+1 < len(data_items):
            if data_items[i][1] == data_items[i+1][1] and switch:
                switch = True
                start = i

            elif switch:
                switch = False

                val = (start+i)/2
                for j in range(start, i+1):
                    tmp[data_items[j][0]] = val + 1

            else:
                tmp[data_items[i][0]] = i + 1
        
        else:
            if switch:

                val = (start+i)/2
                for j in range(start, i+1):
                    tmp[data_items[j][0]] = val + 1

            else:
                tmp[data_items[i][0]] = i + 1
    
    tmp_max = max(tmp)
    tmp_min = min(tmp)
    out = [((tmp[i]-tmp_min)/(tmp_max-tmp_min)) for i in range(len(tmp))]
    
    
    return pd.DataFrame({'tmp': out})

In [13]:
def rank_std_OT(df, colname):
    """ Takes in two inputs: the first a dataframe with two columns: the stat column and the HomeAway column; the second a string which is the column name of the stat column """
    
    tmp1 = df[df['HomeAway'] == 'Home']
    tmp2 = df[df['HomeAway'] == 'Away']
    
    out1 = rank_std_BT(tmp1[colname])
    out2 = rank_std_BT(tmp2[colname])
    
    out = out1.append(out2)
    out.index = list(range(len(out)))
    
    return out

In [14]:
def rank_std_OT_inv(df, colname):
    """ Takes in two inputs: the first a dataframe with two columns: the stat column and the HomeAway column; the second a string which is the column name of the stat column """
    
    tmp1 = df[df['HomeAway'] == 'Home']
    tmp2 = df[df['HomeAway'] == 'Away']
    
    out1 = rank_std_BT_inv(tmp1[colname])
    out2 = rank_std_BT_inv(tmp2[colname])
    
    out = out1.append(out2)
    out.index = list(range(len(out)))
    
    return out

4. Percentage

In [15]:
def per_BT(col):
    """ Takes in input of a dataframe column """
    
    game_sum = col.sum()
    
    return col/game_sum

In [16]:
def per_OT(df, colname):
    """ Takes in input of a dataframe column """
    
    tmp1 = df[df['HomeAway'] == 'Home']
    tmp2 = df[df['HomeAway'] == 'Away']
    
    t1_sum = tmp1[colname].sum()
    t2_sum = tmp2[colname].sum()
    
    out1 = pd.DataFrame(((tmp1[colname]/t1_sum)))
    out2 = pd.DataFrame(((tmp2[colname]/t2_sum)))
    out = out1.append(out2)
    
    return out

5. Percentage followed by Standardisation

In [17]:
# def per_std_BT(col):
#     """ Takes in input of a dataframe column """
    
#     game_sum = col.sum()
    
#     tmp = col/game_sum
    
#     tmp_max = tmp.max()
#     tmp_min = tmp.min()
    
#     return (tmp-tmp_min)/(tmp_max-tmp_min)

In [18]:
# def per_std_BT_inv(col):
#     """ Takes in input of a dataframe column """
    
#     game_sum = col.sum()
    
#     tmp = col/game_sum
    
#     tmp_max = tmp.max()
#     tmp_min = tmp.min()
    
#     return 1 - (tmp-tmp_min)/(tmp_max-tmp_min)

In [19]:
# def per_std_OT(df, colname):
#     """ Takes in two inputs: the first a dataframe with two columns: the stat column and the HomeAway column; the second a string which is the column name of the stat column """
    
#     tmp1 = df[df['HomeAway'] == 'Home']
#     tmp2 = df[df['HomeAway'] == 'Away']
    
#     t1_sum = tmp1[colname].sum()
#     t2_sum = tmp2[colname].sum()
    
#     tmp1 = pd.DataFrame(((tmp1[colname]/t1_sum)))
#     tmp2 = pd.DataFrame(((tmp2[colname]/t2_sum)))

#     t1_max = tmp1[colname].max()
#     t1_min = tmp1[colname].min()
#     t2_max = tmp2[colname].max()
#     t2_min = tmp2[colname].min()
    
#     out1 = pd.DataFrame(((tmp1[colname]-t1_min)/(t1_max-t1_min)))
#     out2 = pd.DataFrame(((tmp2[colname]-t2_min)/(t2_max-t2_min)))
#     out = out1.append(out2)
    
#     return out

In [20]:
# def per_std_OT_inv(df, colname):
#     """ Takes in two inputs: the first a dataframe with two columns: the stat column and the HomeAway column; the second a string which is the column name of the stat column """
    
#     tmp1 = df[df['HomeAway'] == 'Home']
#     tmp2 = df[df['HomeAway'] == 'Away']
    
#     t1_sum = tmp1[colname].sum()
#     t2_sum = tmp2[colname].sum()
    
#     tmp1 = pd.DataFrame(((tmp1[colname]/t1_sum)))
#     tmp2 = pd.DataFrame(((tmp2[colname]/t2_sum)))

#     t1_max = tmp1[colname].max()
#     t1_min = tmp1[colname].min()
#     t2_max = tmp2[colname].max()
#     t2_min = tmp2[colname].min()
    
#     out1 = pd.DataFrame(1-((tmp1[colname]-t1_min)/(t1_max-t1_min)))
#     out2 = pd.DataFrame(1-((tmp2[colname]-t2_min)/(t2_max-t2_min)))
#     out = out1.append(out2)
    
#     return out

6. Percentage followed by Normalisation

In [21]:
# def per_norm_BT(col):
#     """ Takes in input of a dataframe column """
    
#     game_sum = col.sum()
    
#     tmp = col/game_sum
    
#     tmp_mean = tmp.mean()
#     tmp_std = tmp.std()
    
#     return (tmp-tmp_mean)/(tmp_std)

In [22]:
# def per_norm_BT_inv(col):
#     """ Takes in input of a dataframe column """
    
#     game_sum = col.sum()
    
#     tmp = col/game_sum
    
#     tmp_mean = tmp.mean()
#     tmp_std = tmp.std()
    
#     return -1 * (tmp-tmp_mean)/(tmp_std)

In [23]:
# def per_norm_OT(df, colname):
#     """ Takes in two inputs: the first a dataframe with two columns: the stat column and the HomeAway column; the second a string which is the column name of the stat column """
    
#     tmp1 = df[df['HomeAway'] == 'Home']
#     tmp2 = df[df['HomeAway'] == 'Away']
    
#     t1_sum = tmp1[colname].sum()
#     t2_sum = tmp2[colname].sum()
    
#     tmp1 = pd.DataFrame(((tmp1[colname]/t1_sum)))
#     tmp2 = pd.DataFrame(((tmp2[colname]-t2_sum)))

#     t1_mean = tmp1[colname].mean()
#     t1_std = tmp1[colname].std()
#     t2_mean = tmp2[colname].mean()
#     t2_std = tmp2[colname].std()
    
#     out1 = pd.DataFrame(((tmp1[colname]-t1_mean)/(t1_std)))
#     out2 = pd.DataFrame(((tmp2[colname]-t2_mean)/(t2_std)))
#     out = out1.append(out2)
    
#     return out

In [24]:
# def per_norm_OT_inv(df, colname):
#     """ Takes in two inputs: the first a dataframe with two columns: the stat column and the HomeAway column; the second a string which is the column name of the stat column """
    
#     tmp1 = df[df['HomeAway'] == 'Home']
#     tmp2 = df[df['HomeAway'] == 'Away']
    
#     t1_sum = tmp1[colname].sum()
#     t2_sum = tmp2[colname].sum()
    
#     tmp1 = pd.DataFrame(((tmp1[colname]/t1_sum)))
#     tmp2 = pd.DataFrame(((tmp2[colname]-t2_sum)))

#     t1_mean = tmp1[colname].mean()
#     t1_std = tmp1[colname].std()
#     t2_mean = tmp2[colname].mean()
#     t2_std = tmp2[colname].std()
    
#     out1 = pd.DataFrame(-1*((tmp1[colname]-t1_mean)/(t1_std)))
#     out2 = pd.DataFrame(-1*((tmp2[colname]-t2_mean)/(t2_std)))
#     out = out1.append(out2)
    
#     return out

**2. Running functions on every file**

In [30]:
filelist = os.listdir(f'./Data/OriginalData_AddDerived')
filelist.sort()
filelist = filelist[1:]

# Remove the first file (an ipynb checkpoint file)

In [26]:
# First make dictionaries which direct the flow of control as to whether a column shoul receive manipulation and whether it should be positive or inversed direction.

# Unchanged
Orig = ['Player', 'Winloss', 'Brownlow Votes', 'HomeAway']

# Normal manipulation 
normal = ['Kicks', 'Handballs', 'Disposals', 'Marks', 'Goals', 'Behinds', 'Tackles', 'Hitouts', 'Goal Assists', 'Inside 50s', 
               'Clearances', 'Rebound 50s', 'Frees For', 'Contested Possessions', 'Uncontested Possessions', 
               'Effective Disposals', 'Contested Marks', 'Marks Inside 50', 'One Percenters', 'Bounces', 'Centre Clearances', 
               'Stoppage Clearances', 'Score Involvements', 'Metres Gained', 'Intercepts', 'Tackles Inside 50', 'Time On Ground %', 'Uncontested Marks',
               'Marks Outside 50', 'Tackles Outside 50', 'Behind Assists', 'Effective Disposals', 'Ineffective Disposals']

# Inversed manipulation
invert = ['Clangers', 'Turnovers', 'Frees Agains']

In [32]:
%time

for file in filelist:
    
    df = pd.read_csv(f'./Data/OriginalData_AddDerived/{file}')
    
    
    std = pd.DataFrame()
    
    norm = pd.DataFrame()
    
    # norm_std = pd.DataFrame()
    
    rank_std = pd.DataFrame()
    
    # per = pd.DataFrame()
    
#     per_std = pd.DataFrame()
    
    # per_norm = pd.DataFrame()
    
    
    for column in df.columns:
        
        if column in Orig:
            std[column] = list(df[column])
            
            # norm[column] = list(df[column])
            
            # norm_std[column] = list(df[column])
            
            rank_std[column] = list(df[column])
            
            # per[column] = list(df[column])
            
#             per_std[column] = list(df[column])
            
            # per_norm[column] = list(df[column])
            

        elif column in normal:
            std[f'{column} BTS'] = std_BT(df[column])
            std[f'{column} OTS'] = std_OT(df[[column, 'HomeAway']], column)
            
            # norm[f'{column} BTN'] = norm_BT(df[column])
            # norm[f'{column} OTN'] = norm_OT(df[[column, 'HomeAway']], column)
            
            # norm_std[f'{column} BTNS'] = norm_std_BT(df[column])
            # norm_std[f'{column} OTNS'] = norm_std_OT(df[[column, 'HomeAway']], column)
            
            rank_std[f'{column} BTRS'] = rank_std_BT(df[column])
            rank_std[f'{column} OTRS'] = rank_std_OT(df[[column, 'HomeAway']], column)
            
            # per[f'{column} BTP'] = per_BT(df[column])
            # per[f'{column} OTP'] = per_OT(df[[column, 'HomeAway']], column)
            
#             per_std[f'{column} BTPS'] = per_std_BT(df[column])
#             per_std[f'{column} OTPS'] = per_std_OT(df[[column, 'HomeAway']], column)
            
            # per_norm[f'{column} BTPN'] = per_norm_BT(df[column])
            # per_norm[f'{column} OTPN'] = per_norm_OT(df[[column, 'HomeAway']], column)
            
            
        elif column in invert:
            std[f'{column} BTS'] = std_BT_inv(df[column])
            std[f'{column} OTS'] = std_OT_inv(df[[column, 'HomeAway']], column)
            
            # norm[f'{column} BTN'] = norm_BT_inv(df[column])
            # norm[f'{column} OTN'] = norm_OT_inv(df[[column, 'HomeAway']], column)
            
            # norm_std[f'{column} BTNS'] = norm_std_BT_inv(df[column])
            # norm_std[f'{column} OTNS'] = norm_std_OT_inv(df[[column, 'HomeAway']], column)
            
            rank_std[f'{column} BTRS'] = rank_std_BT_inv(df[column])
            rank_std[f'{column} OTRS'] = rank_std_OT_inv(df[[column, 'HomeAway']], column)
            
            # per[f'{column} BTP'] = per_BT(df[column])
            # per[f'{column} OTP'] = per_OT(df[[column, 'HomeAway']], column)
            
#             per_std[f'{column} BTPS'] = per_std_BT_inv(df[column])
#             per_std[f'{column} OTPS'] = per_std_OT_inv(df[[column, 'HomeAway']], column)
            
            # per_norm[f'{column} BTPN'] = per_norm_BT_inv(df[column])
            # per_norm[f'{column} OTPN'] = per_norm_OT_inv(df[[column, 'HomeAway']], column)
        
        
        else:
            # Error mechanism
            print(column)
    
    
    std.to_csv(f'./Data/StandardisedData/{file.strip("(O).csv")} (S).csv', index=False)
    
    # norm.to_csv(f'./Data/NormalisedData/{file.strip("(O).csv")} (N).csv', index=False)
    
    # norm_std.to_csv(f'./Data/NormalisedStandardisedData/{file.strip("(O).csv")} (NS).csv', index=False)
    
    rank_std.to_csv(f'./Data/RankStandardisedData/{file.strip("(O).csv")} (RS).csv', index=False)
    
    # per.to_csv(f'./Data/PercentageData/{file.strip("(O).csv")} (P).csv', index=False)
    
#     per_std.to_csv(f'./Data/PercentageStandardisedData/{file.strip("(O).csv")} (PS).csv', index=False)
    
    # per_norm.to_csv(f'./Data/PercentageNormalisedData/{file.strip("(O).csv")} (PN).csv', index=False)
    

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.25 µs


## Note: A few improvements could be made on this notebook: ##

*1. the Rank manipulation process is mostly done using Python built-in types. This may cause the process to be rather slow. May be faster if a Pandas built in function could be used to achieve the same means*